In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import numpy as np
np.random.seed(132)
from functools import lru_cache

import sys

CODE_PATH = '../code'

sys.path.append(CODE_PATH)
import functions


from sklearn.datasets import load_iris, load_boston, load_breast_cancer, load_wine, load_digits
from scipy.optimize import minimize
from tqdm import tqdm
%pylab inline

Populating the interactive namespace from numpy and matplotlib


бостон - градиент уходит в 0  
ирис - спустя пару итераций в 0

In [3]:
dataset = load_breast_cancer()
# dataset = load_boston()
df = pd.DataFrame(dataset['data'])
target = dataset['target']
df = (df - df.mean())/(df.max() - df.min())
df0 = df.copy()
print(df.shape)
print(target)
df.head()

(569, 30)
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 1 0 0 0 0 0 0 0 0 1 0 1 1 1 1 1 0 0 1 0 0 1 1 1 1 0 1 0 0 1 1 1 1 0 1 0 0
 1 0 1 0 0 1 1 1 0 0 1 0 0 0 1 1 1 0 1 1 0 0 1 1 1 0 0 1 1 1 1 0 1 1 0 1 1
 1 1 1 1 1 1 0 0 0 1 0 0 1 1 1 0 0 1 0 1 0 0 1 0 0 1 1 0 1 1 0 1 1 1 1 0 1
 1 1 1 1 1 1 1 1 0 1 1 1 1 0 0 1 0 1 1 0 0 1 1 0 0 1 1 1 1 0 1 1 0 0 0 1 0
 1 0 1 1 1 0 1 1 0 0 1 0 0 0 0 1 0 0 0 1 0 1 0 1 1 0 1 0 0 0 0 1 1 0 0 1 1
 1 0 1 1 1 1 1 0 0 1 1 0 1 1 0 0 1 0 1 1 1 1 0 1 1 1 1 1 0 1 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 1 1 1 1 1 1 0 1 0 1 1 0 1 1 0 1 0 0 1 1 1 1 1 1 1 1 1 1 1 1
 1 0 1 1 0 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 0 1 0 1 1 1 1 0 0 0 1 1
 1 1 0 1 0 1 0 1 1 1 0 1 1 1 1 1 1 1 0 0 0 1 1 1 1 1 1 1 1 1 1 1 0 0 1 0 0
 0 1 0 0 1 1 1 1 1 0 1 1 1 1 1 0 1 1 1 0 1 1 0 0 1 1 1 1 1 1 0 1 1 1 1 1 1
 1 0 1 1 1 1 1 0 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 0 1 0 0 1 0 1 1 1 1 1 0 1 1
 0 1 0 1 1 0 1 0 1 1 1 1 1 1 1 1 0 0 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 0 1
 1 1 1 1 1 1 0 

,0,1,2,3,4,5,6,7,8,9,...,20,21,22,23,24,25,26,27,28,29
0,0.182815,-0.301307,0.213053,0.146813,0.198968,0.531437,0.495081,0.487976,0.306758,0.335139,...,0.324113,-0.222474,0.385173,0.279792,0.196998,0.399079,0.351207,0.518192,0.335156,0.229268
1,0.304923,-0.051392,0.282848,0.284671,-0.104905,-0.078833,-0.004450,0.105621,0.000193,-0.129057,...,0.310239,-0.060427,0.256680,0.264308,-0.056585,-0.065649,-0.024432,0.245339,-0.029717,0.033282
2,0.263274,0.066295,0.262808,0.232497,0.119524,0.170416,0.254453,0.392549,0.129991,-0.059132,...,0.259723,-0.003924,0.225304,0.203602,0.079452,0.165163,0.142341,0.441216,0.140399,0.023837
3,-0.128132,0.036874,-0.099434,-0.114014,0.416536,0.550761,0.357546,0.279726,0.396657,0.729621,...,-0.048353,0.021929,-0.041791,-0.076898,0.511335,0.593799,0.331239,0.491044,0.736693,0.584115
4,0.291671,-0.167388,0.298051,0.272369,0.035567,0.087292,0.255859,0.275253,-0.001323,-0.083564,...,0.223081,-0.240065,0.223810,0.170669,0.033226,-0.047797,0.102086,0.164583,-0.105806,-0.047001


In [4]:
num_nan_cols = 2
nan_fraction = 0.3
nan_cols = np.random.random_integers(0, df.shape[1] - 1, num_nan_cols)
# print(df.isnull().mean())
for col in set(nan_cols):
    df.loc[df.sample(int(nan_fraction * len(df))).index, col] = np.nan
# print(df.isnull().mean())
# print(df.isnull().mean())

/Users/kek/mlenv/lib/python3.6/site-packages/ipykernel_launcher.py:3: DeprecationWarning: This function is deprecated. Please call randint(0, 29 + 1) instead
  This is separate from the ipykernel package so we can avoid doing imports until


In [5]:
nan_coords = np.array(np.where(df.isnull().values)).T
print('Num nan places: {}'.format(nan_coords.shape[0]))

Num nan places: 340


Считаем по каждой паре i_1 i_2 из coords градиент

In [6]:
df1 = df.loc[:, df.isnull().sum() == 0]
df2 = df.fillna(df.mean())
print(df1.shape, df2.shape)
arr_nan = df.values
arr_raw = df0.values # исходные
arr_known = df1.values # суженные до известных признаков
arr_pred = df2.values # текущие предсказанные 
# print(arr0.shape, arr1.shape, arr2.shape)
# print(np.isnan(arr0).sum(), np.isnan(arr1).sum(), np.isnan(arr2).sum())

(569, 28) (569, 30)


# Пилим необходимые функции

df - исходная с прочерками    
df0 - копия исходной таблицы без прочерков  
df1 - сокращенная до ненулевых признаков  
df2 - исходная с восстановленными  
arr0,arr1,arr2 аналогично

In [7]:
def sq(a):
    return np.dot(a, a)

n = len(df0)

def get_known_table():
    ro_known_table = np.zeros((n, n))
    for i in tqdm(range(n)):
        for j in range(i, n):
            idx_nan_i = np.isnan(arr_nan[i])
            idx_nan_j = np.isnan(arr_nan[j])
            idx = ~idx_nan_i & ~idx_nan_j
            x_i = arr_nan[i, idx]
            x_j = arr_nan[j, idx]
            ro_known_table[i, j] = ro_known_table[j, i] = sq(x_i - x_j)
    return ro_known_table

def get_ro_1_unknown_table():
    ro_1_unknown_table = np.zeros((n, n))
    #это ro прибавить там где 1 компонента
    for i in tqdm(range(n)):
        for j in range(i, n):
            idx_nan_i = np.isnan(arr_nan[i])
            idx_nan_j = np.isnan(arr_nan[j])
            idx = (idx_nan_i & ~idx_nan_j) | (~idx_nan_i & idx_nan_j)
            x_i = arr_pred[i, idx]
            x_j = arr_pred[j, idx]
            ro_1_unknown_table[i, j] = ro_1_unknown_table[j, i] = sq(x_i - x_j)
    return ro_1_unknown_table

# по всем признакам
def get_ro_2_unknown_table():
    ro_2_unknown_table = np.zeros((n, n))
    for i in tqdm(range(n)):
        for j in range(i, n):
            idx_nan_i = np.isnan(arr_nan[i])
            idx_nan_j = np.isnan(arr_nan[j])
            idx = (idx_nan_i & idx_nan_j)
            x_i = arr_pred[i, idx]
            x_j = arr_pred[j, idx]
            ro_2_unknown_table[i, j] = ro_2_unknown_table[j, i] = sq(x_i - x_j)
    return ro_2_unknown_table

In [8]:
ro_known_table = get_known_table()
ro_1_unknown_table = get_ro_1_unknown_table()
ro_2_unknown_table = get_ro_2_unknown_table()

100%|██████████| 569/569 [00:05<00:00, 95.12it/s]


In [9]:
def F1(k_arr):
    '''
    k - коэффициент при втором расстоянии
    '''
    f = 0
    for i in tqdm(range(n)):
        for j in range(i+1, n):
            ro1 = ro_known_table[i, j] # известно оба 
            ro2 = ro_1_unknown_table[i, j] # известен 1 
            ro3 = ro_2_unknown_table[i, j] # не известно ниче
            
            r1 = np.sqrt(ro1 + ro2 + ro3) #по всем признакам(р)
            r2 = np.sqrt(ro1) # по известным признакам(р+)
            f += (r1 - k_arr[i, j]*r2)**2
    return f

def F2(k_arr):
    '''
    k - коэффициент при втором расстоянии
    '''
    f = 0
    for i in tqdm(range(n)):
        for j in range(i+1, n):
            ro1 = ro_known_table[i, j] # известно оба 
            ro2 = ro_1_unknown_table[i, j] # известен 1 
            ro3 = ro_2_unknown_table[i, j] # не известно ниче
            
            r1 = np.sqrt(ro1 + ro2 + ro3)
            r2 = np.sqrt(ro1 + ro2)
            f += (r1 - k_arr[i, j]*r2)**2
    return f

In [10]:
def grad_f2(i, j, k_arr):
    '''
    Считает градиент по y_i,j
    '''
    s = 0
    for i1 in range(n):
        ro1 = ro_known_table[i, i1]
        if ro1 <= 0:
            continue
        if i1 == i:
            continue
        ro2 = ro_1_unknown_table[i, i1]
        ro3 = ro_2_unknown_table[i, i1]
        
        r1 = np.sqrt(ro1 + ro2 + ro3) #по всем признакам(р)
        r2 = np.sqrt(ro1 + ro2) # по известным признакам(р+)
        if r1 > 0 and r2 > 0:
            s += 2 * (r1 - k_arr[i1, i]*r2)
            delta_x1 = (arr_pred[i, j] - arr_pred[i1, j])
            if not np.isnan(arr_nan[i1, j]):
                delta_x2 = delta_x1
            else:
                delta_x2 = 0
            delta_s = (r2 - k*r1)*(delta_x1/r2 - k_arr[i1, i]*delta_x2/r1)
            s += delta_s
    return s

def get_full_grad_f2(nan_coords, k_arr):
    grad = []
    for i, j in nan_coords:
        grad.append(grad_f2(i, j, k_arr))
    return np.array(grad)

In [11]:
def grad_f1(i, j, k_arr):
    '''
    Считает градиент по y_i,j
    '''
    s = 0
    for i1 in range(n):
        ro1 = ro_known_table[i, i1]
        if ro1 <= 0:
            continue
        if i1 == i:
            continue
        ro2 = ro_1_unknown_table[i, i1]
        ro3 = ro_2_unknown_table[i, i1]
        r1 = np.sqrt(ro1 + ro2 + ro3) #по всем признакам(р)
        r2 = np.sqrt(ro1) # по известным признакам(р+)
        s += 2 * (1 - k_arr[i1, i]*r2/r1) * (arr_pred[i, j] - arr_pred[i1, j])
    return s

def get_full_grad_f1(nan_coords, k_arr):
    grad = []
    for i, j in nan_coords:
        grad.append(grad_f1(i, j, k_arr))
    return np.array(grad)

from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error

def get_mae(arr1, arr2, nan_coords):
    vec1 = []
    vec2 = []
    for j, (x,y) in enumerate(nan_coords):
        vec1.append(arr1[x, y])
        vec2.append(arr2[x, y])
    return mean_absolute_error(vec1, vec2)

def get_msqe(arr1, arr2, nan_coords):
    vec1 = []
    vec2 = []
    for j, (x,y) in enumerate(nan_coords):
        vec1.append(arr1[x, y])
        vec2.append(arr2[x, y])
    return mean_squared_error(vec1, vec2)

Тестовый прогон

# Iterative process

In [12]:
k_arr = np.ones((n, n))
c = F1(k_arr)

100%|██████████| 569/569 [00:01<00:00, 314.67it/s]


In [1]:
mae = functions.get_mae(arr_raw, arr_pred, nan_coords)
mae

NameError: name 'functions' is not defined

In [14]:
cs = []
maes = []
msqes = []
accs = []

mae = functions.get_mae(arr_raw, arr_pred, nan_coords)
msqe = functions.get_msqe(arr_raw, arr_pred, nan_coords)
acc = functions.get_acc(arr_pred, target)

cs.append(c)
maes.append(mae)
msqes.append(msqe)
accs.append(acc)

In [15]:
alpha = 0.001
max_iter = 30
print('\tIteration {} //  c {:.6} // mae {:.6} // rmse {:.6} // acc {}'.format(0, c, mae, msqe, acc))
for i in range(max_iter):
    grad = get_full_grad_f1(nan_coords, k_arr)
    for j, (x,y) in enumerate(nan_coords):
        arr_pred[x, y] = arr_pred[x, y] - alpha*grad[j]
        
    ro_1_unknown_table = get_ro_1_unknown_table()
    ro_2_unknown_table = get_ro_2_unknown_table()
    c = F1(k_arr)
    mae = functions.get_mae(arr_raw, arr_pred, nan_coords)
    msqe = functions.get_msqe(arr_raw, arr_pred, nan_coords)
    acc = functions.get_acc(arr_pred, target)
    
    accs.append(acc)
    cs.append(c)
    maes.append(mae)
    msqes.append(msqe)
    print('\tIteration {} // c {:.6} // mae {:.6} // rmse {:.6} // acc {}'.format(i+1, c,mae, msqe, acc))

	Iteration 0 //  c 89.5749 // mae 0.120803 // rmse 0.0230583 // acc 0


100%|██████████| 569/569 [00:01<00:00, 373.82it/s]


	Iteration 1 // c 88.9169 // mae 0.119834 // rmse 0.0227454 // acc 0


100%|██████████| 569/569 [00:01<00:00, 376.01it/s]


	Iteration 2 // c 88.3199 // mae 0.118917 // rmse 0.0224506 // acc 0


...

	Iteration 14 // c 84.1445 // mae 0.110998 // rmse 0.0199427 // acc 0


100%|██████████| 569/569 [00:01<00:00, 351.46it/s]


	Iteration 15 // c 83.9521 // mae 0.110531 // rmse 0.0197962 // acc 0


  6%|▌         | 32/569 [00:00<00:10, 49.08it/s]


KeyboardInterrupt: 

In [ ]:
grad = get_full_grad_f1(nan_coords, k_arr)

In [ ]:
grad